# Capstone Project - Part I
by **Lukas Schumi**

## 1. Introduction

### 1.1 Background

I currently work for the company **XXL Sports & Outdoor GMBH**, a Scandinavian sports retailer which is expanding in Austria. The logical starting point was the capital of Austria, Vienna, which inherits almost a quarter of the population of the entire country. It was also a good starting point, for Vienna offers a wide variety for those of us, who want to stay active and enroll in any kind of sportive activity. 
But there is also a lot of competition on the sporting goods market. And with 5 Stores in 3 major cities and a 6th one just weeks from opening, expansions is always a hot topic. 
Therefore analyzing locations is key for finding the perfect position for a new store. 


### 1.2 Problem

Austrias national sport is skiing and there are many first-class skiing areas especially in the western regions of the country. In my opinion, when it comes to sport, the skiing market might be the hardest to enter in this country, for there are countless long-established specialized shops.  
So I think it might be smarter to focus on the eastern regions until the brand of XXL made a name for itself as a reliable retailer. And where better to start a comprehensive branch network than in the capital?  
So in the following work I focus on the city of Vienna and its immediate surroundings. Searching an optimal location for a new store can be challenging. You have to find the right balance between a place with too much competition and one that's too remote to reach. To narrow the possibilities down, I looked at locations where places are nearby, where sport equipment is needed. The thought behind this is, that if someone is at such a place, say a gym or a beach, and forgot his gear or something breaks during usage, there might be a bigger chance, if a sports good shop is nearby, that said person buys new equipment instead of going home and picking it up or ordering it online. Ideally there should also be some kind of public transport nearby.






### 1.3 Interest

This project is meant for stakeholders at **XXL Sports & Outdoor GMBH** as aid when searching for the next location to open a new store.

## 2. Data acquisition

### 2.1 Data source

For this project I got locations for sport facilities as well as the places of competing stores in Vienna from the **Foursquare API**.
The locations of the XXL stores are from the Homepage https://www.xxlsports.at/store-finder and for comparisons I used the official city map of Vienna https://www.wien.gv.at/stadtplan/en/. From the last one I took the coordinates of the St. Stephen's Cathedral ("Stephansdom") which set as center of the city. 

### 2.2 Feature selection

* I will create two datasets using the **Foursquare API**. One for sporting goods shops in Vienna, containing information about name, location (address as well as geographical coordinates) and associated category. The second dataset will contain the same information about sport facilities.

* The data collected from Foursquare will be saved as **csv files** and then read into **pandas dataframes**.

* To visualize the proximity of the sporting facilities to the shops and compare them to existing XXL stores I will create a **Folium map**.

### 2.3 Logging

In [1]:
import pandas as pd
import requests

Defining Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'XXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXX' # your Foursquare Secret
VERSION = '20201504'

Longitude and latitude of center of Vienna using https://www.wien.gv.at/stadtplan/en/

In [3]:
vienna_lat = 48.208489
vienna_long = 16.373156

Searching in a radius of 15km around the St. Stephen's Cathedral

In [4]:
radius = 15000

#### 2.3.1 Sporting good shops

Finding all categories of Sporting goods shop

In [5]:
Sporting_Goods_Shop = '4bf58dd8d48988d1f2941735' 
Skate_Shop = '5bae9231bedf3950379f89d2'
Ski_Shop = '56aa371be4b08b9a8d573566'
Supplement_Shop = '5744ccdfe4b0c0459246b4cd'
Bike_Shop = '4bf58dd8d48988d115951735'

shops = ['4bf58dd8d48988d1f2941735', '5bae9231bedf3950379f89d2', '56aa371be4b08b9a8d573566', '5744ccdfe4b0c0459246b4cd', '4bf58dd8d48988d115951735']

In [6]:
categories_shops = []

for x in shops:
    shop_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, vienna_lat, vienna_long, x, radius)
    res = requests.get(shop_url).json()
    categories_shops.append(res)

In [7]:
grouplist = []

for k in range (0, len(categories_shops)):
    
    x = categories_shops[k]['response']['groups']
    grouplist.append(x)

listofall = []

for a in range (0, len(grouplist)):
    b = grouplist[a][0]['items']
    for c in range (0, len(b)):
        if len(b) != 0:
            z = b[c]
            listofall.append(z)

In [8]:
allshops = []

for element in range (0, len(listofall)):
    c = listofall[element]['venue']
    allshops.append(c)

In [9]:
venue_namelist = []
venue_adresslist = []
venue_catlist = []
venue_latlist = []
venue_lnglist = []

for i in range(0, len(allshops)):
    venues_name = allshops[i]['name']
    venues_adress = allshops[i]['location']['formattedAddress']
    venues_category = allshops[i]['categories'][0]['pluralName']
    venues_lat = allshops[i]['location']['lat']
    venues_lng = allshops[i]['location']['lng']
    venue_namelist.append(venues_name)
    venue_adresslist.append(venues_adress)
    venue_catlist.append(venues_category)
    venue_latlist.append(venues_lat)
    venue_lnglist.append(venues_lng)

In [10]:
data = {'Name':venue_namelist, 'Adress': venue_adresslist, 'Cat': venue_catlist, 'Lat': venue_latlist, 'Lng': venue_lnglist }
df = pd.DataFrame(data)

df.head()

,Name,Adress,Cat,Lat,Lng
0,Laufsport Blutsch,"[Liniengasse 24, 1060 Wien, Österreich]",Sporting Goods Shops,48.192410,16.344130
1,bergfuchs,"[Kaiserstraße 15, Österreich]",Sporting Goods Shops,48.198428,16.341169
2,Foot Locker,"[Mariahilfer Str. 104, 1070 Wien, Österreich]",Sporting Goods Shops,48.196810,16.344213
3,HANGLOOSE Boardsports Store,"[Favoritenstr. 28 (Mayerhofgasse), 1040 Wien, ...",Sporting Goods Shops,48.192957,16.370626
4,Blue Tomato Shop Wien,"[Neubaugasse 3, 1070 Wien, Österreich]",Board Shops,48.198623,16.349967


In [11]:
df.shape

(68, 5)

In [12]:
df.to_csv(r'~/XXXXX/shops.csv', index = False)

#### 2.3.2 Sport facilities

In [13]:
Gym_Fitness = '4bf58dd8d48988d175941735' 
# subcategories of 'gym'
Boxing_Gym ='52f2ab2ebcbc57f1066b8b47'
Climbing_Gym = '503289d391d4c4b30a586d6a'
Cycle_Studio = '52f2ab2ebcbc57f1066b8b49'
Gym_Pool = '4bf58dd8d48988d105941735'
Gymnastics_Gym = '52f2ab2ebcbc57f1066b8b48'
Gym = '4bf58dd8d48988d176941735'
Martial_Arts_Dojo = '4bf58dd8d48988d101941735'
Outdoor_Gym = '58daa1558bbb0b01f18ec203'
Pilates_Studio = '5744ccdfe4b0c0459246b4b2'
Track = '4bf58dd8d48988d106941735'
Weight_Loss_Center = '590a0744340a5803fd8508c3'
Yoga_Studio = '4bf58dd8d48988d102941735'

Basketball_Court = '4bf58dd8d48988d1e1941735'  
Hockey_Field = '4f452cd44b9081a197eba860'
Skate_Park = '4bf58dd8d48988d167941735'
Skating_Rink = '4bf58dd8d48988d168941735'
Soccer_Field = '4cce455aebf7b749d5e191f5'
Sports_Club = '52e81612bcbc57f1066b7a2e'
Tennis_Court = '4e39a956bd410d7aed40cbc3'
Volleyball_Court = '4eb1bf013b7b6f98df247e07'
Bathing_Area = '52e81612bcbc57f1066b7a28'
Beach = '4bf58dd8d48988d1e2941735'
Ski_Area = '4bf58dd8d48988d1e9941735'
College_Gym = '4bf58dd8d48988d1b2941735'
College_Stadium = '4bf58dd8d48988d1b4941735'

In [14]:
categorylist = [Boxing_Gym, Climbing_Gym, Cycle_Studio, Gym_Pool, Gymnastics_Gym, Gym, Martial_Arts_Dojo, Outdoor_Gym, Pilates_Studio, Track, Weight_Loss_Center,Yoga_Studio, Basketball_Court, Hockey_Field , Skate_Park , Skating_Rink , Soccer_Field , Sports_Club , Tennis_Court , Volleyball_Court ,Bathing_Area , Beach, Ski_Area , College_Gym ,College_Stadium]

In [15]:
resultlist =[]

for x in categorylist:
    url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, vienna_lat, vienna_long, x, radius)
    res = requests.get(url2).json()
    resultlist.append(res)
    
len(resultlist)

25

In [16]:
venlist = []
for k in range (0, len(resultlist)):
    venues = resultlist[k]['response']['groups'][0]['items']
    venlist.append(venues)

In [17]:
loclist = []
for x in range (0, len(venlist)):
    y = venlist[x]
    for z in range (0, len(y)):
        location = y[z]
        loclist.append(location)

In [18]:
len(loclist)

329

In [19]:
div_list = []
div_dict = []

for i in range (0, len(loclist)):
    if type(loclist[i]) == dict:
        div_dict.append(loclist[i])
    else:
        div_list.append(loclist[i])

In [20]:
print(len(div_list), len(div_dict))

0 329


In [21]:
namelist = []
adresslist = []
catlist = []
latlist = []
lnglist = []

for z in range (0, len(loclist)):
    location_name = loclist[z]['venue']['name']
    location_adress = loclist[z]['venue']['location']['formattedAddress']
    location_category = loclist[z]['venue']['categories'][0]['pluralName'] 
    location_lat = loclist[z]['venue']['location']['labeledLatLngs'][0]['lat']
    location_lng = loclist[z]['venue']['location']['labeledLatLngs'][0]['lng']
    namelist.append(location_name)
    adresslist.append(location_adress)
    catlist.append(location_category)
    latlist.append(location_lat)
    lnglist.append(location_lng)

In [22]:
len(namelist)

329

In [23]:
locations = {'Name':namelist, 'Adress': adresslist, 'Category': catlist, 'Lat': latlist, 'Lng': lnglist }
df_location = pd.DataFrame(locations)
df_location.head(10)

,Name,Adress,Category,Lat,Lng
0,Ostermann Kickboxen,"[Gärtnergasse 4, 1030 Wien, Österreich]",Athletics & Sports,48.206592,16.388362
1,Octagon Kickboxing,[Österreich],Boxing Gyms,48.196937,16.344497
2,Boxteam Vienna,"[Stadlauerstrasse 41a, 1220 Wien, Österreich]",Boxing Gyms,48.232381,16.456469
3,Gym 23,"[Anton-Baumgartner-Straße 40, 1230 Wien, Öster...",Gyms or Fitness Centers,48.152249,16.318870
4,OeAV Edelweiss-Center,"[Walfischg. 12, 1010 Wien, Österreich]",Climbing Gyms,48.202806,16.372683
5,boulderbar Vienna,"[Hannovergasse 21, 1200 Wien, Österreich]",Climbing Gyms,48.231446,16.367896
6,Kletterhalle Wien,"[Erzherzog-Karl-Straße 108, 1220 Wien, Österre...",Climbing Gyms,48.230189,16.450644
7,Sportzentrum Marswiese,"[Neuwaldegger Straße 57a, 1170 Wien, Österreich]",Climbing Gyms,48.239980,16.276590
8,SÜDWAND Kletterzentrum,"[Plättenstr. 2, 2380 Perchtoldsdorf, Österreich]",Climbing Gyms,48.123461,16.266499
9,Kletterwand am Flakturm,"[Esterházypark (Fritz-Grünbaum-Platz 1), 1060 ...",Climbing Gyms,48.197643,16.352663


In [24]:
df_location.shape

(329, 5)

In [25]:
df_location.to_csv(r'~/XXXX/locations.csv', index = False)